## Removing Dark Current and Bias Iteratively

Below we demonstrate how a 200x200 image with a few stars, plus noise, dark current and bias, changes when the dark current and the bias are removed repeatedly, each time with fresh random noise.

In [ ]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

import image_sim as imsim
from convenience_functions import show_image

In [ ]:
plt.style.use('guide.mplstyle')

In [ ]:
image = np.zeros([500, 500])
gain = 1.0
noise_amount = 150

In [ ]:
gain = 1.0
star_exposure = 30.0
dark_exposure = 60.0
dark = 0.1
sky_counts = 20
bias_level = 1100
read_noise_electrons = 1500
max_star_counts = 10000

In [ ]:
flat = imsim.sensitivity_variations(image)

In [ ]:
real_stars      = imsim.stars(image, 10, max_counts=max_star_counts)
realistic_stars = (real_stars +
                   imsim.dark_current(image, dark, star_exposure, gain=gain, hot_pixels=True) +
                   imsim.bias(image, bias_level, realistic=True) +
                   imsim.read_noise(image, read_noise_electrons, gain=gain)
                  )

In [ ]:
def bias_and_dark(image):
    bias_with_noise = (imsim.bias(image, bias_level, realistic=True) +
                       imsim.read_noise(image, read_noise_electrons, gain=gain))
    dark_with_noise = (imsim.bias(image, bias_level, realistic=True) +
                             imsim.dark_current(image, dark, dark_exposure, gain=gain, hot_pixels=True) +
                             imsim.read_noise(image, read_noise_electrons, gain=gain))
    return (bias_with_noise, dark_with_noise)

Below we generate three rows of images. The first row shows just the stars on the left, and the stars plus noise, dark current and bias on the right. In the second and third row, we remove the dark current and bias, first once, then twice and so on. Each removal also adds fresh noise, so there is a balance between removing too little and removing too much.

In [ ]:
percu=99.9
fig, axes = plt.subplots(3, 2, figsize=(60,50))

show_image(real_stars, cmap='gray', percu=percu, fig=fig, ax=axes[0,0])
calibrated_stars = realistic_stars
show_image(calibrated_stars, cmap='gray', percu=percu, fig=fig, ax=axes[0,1])

for row in range(1,3):
    for col in range(2):
        ax = axes[row,col]
        (f_bias,f_dark) = bias_and_dark(image=image)
        scaled_dark_current = star_exposure * (f_dark - f_bias) / dark_exposure
        calibrated_stars = (calibrated_stars - f_bias - scaled_dark_current) / flat

        show_image(calibrated_stars, cmap='gray', percu=percu, fig=fig, ax=ax)